# Bitcoin Direction Prediction with 2D CNN
- In this problem, we will integrate technical analysis indicators into an original time series, which will help us convert the original series into 2D images.
- We will use the following technical indicators from the TA-Lib library in Python: MACD, RSI, CMO, MOM, Bollinger Bands, and SMA. 
- Technical analysis indicators are financial indicators that guide traders about the market.
- We will use AAPL close prices from the "yfinance" library. The training period is 2021-2022, and the test period is 2022-2023.
- We will use the historical 10-day closing price, build up a 6x6 image by calculating technical indicators, and predict the direction for the next day, whether the price will go up or down.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import callbacks
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [2]:
# Download AAPL close price data
data = yf.download("AAPL", start="2021-01-01", end="2023-01-01")
data = data[['Close']].reset_index()

# Split into train (2021-2022) and test (2022-2023)
train_df = data[(data['Date'] >= "2021-01-01") & (data['Date'] <= "2022-01-01")]
test_df = data[(data['Date'] > "2022-01-01") & (data['Date'] <= "2022-12-31")]

# Drop NaNs (if applicable)
train_df = train_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

[*********************100%***********************]  1 of 1 completed

Train shape: (252, 2)
Test shape: (251, 2)


In [3]:
train_df.head()

Price,Date,Close
Ticker,,AAPL
0,2021-01-04 00:00:00+00:00,129.410004
1,2021-01-05 00:00:00+00:00,131.009995
2,2021-01-06 00:00:00+00:00,126.599998
3,2021-01-07 00:00:00+00:00,130.919998
4,2021-01-08 00:00:00+00:00,132.050003


In [4]:
# Generate features, such as MACD, RSI, CMO, MOM, Bollinger Bands, and SMA, by using TA-Lib
def calculate_indicators(data):
    # Ensure the 'Close' column is a NumPy array
    close_prices = data['Close'].values.flatten()  # Convert to 1D array

    # Calculate indicators
    data['RSI'] = talib.RSI(close_prices, timeperiod=14)
    data['CMO'] = talib.CMO(close_prices, timeperiod=14)
    data['MOM'] = talib.MOM(close_prices, timeperiod=10)
    macd, macd_signal, _ = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
    data['MACD'] = macd - macd_signal  # Difference between MACD and Signal line
    
    # Bollinger Bands (adds three new columns)
    data['BB_Upper'], data['BB_Middle'], data['BB_Lower'] = talib.BBANDS(
        close_prices, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
    )
    
    # Simple Moving Average (SMA)
    data['SMA'] = talib.SMA(close_prices, timeperiod=14)

    return data

# Apply to train and test datasets
train_data = calculate_indicators(train_df)
test_data = calculate_indicators(test_df)

# Drop rows with missing values
train_data = train_data.dropna().reset_index(drop=True)
test_data = test_data.dropna().reset_index(drop=True)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (219, 10)
Test shape: (218, 10)


In [5]:
train_data.head()

Price,Date,Close,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA
Ticker,,AAPL,,,,,,,,
0,2021-02-22 00:00:00+00:00,126.000000,35.842782,-28.314437,-10.759995,-2.001957,143.672556,135.1415,126.610444,133.678571
1,2021-02-23 00:00:00+00:00,125.860001,35.632885,-28.734230,-11.050003,-2.012881,142.948565,134.2885,125.628436,133.026428
2,2021-02-24 00:00:00+00:00,125.349998,34.832645,-30.334709,-10.659996,-1.958668,141.887017,133.3980,124.908983,132.412857
3,2021-02-25 00:00:00+00:00,120.989998,28.864578,-42.270843,-14.400002,-2.107705,141.477387,132.3445,123.211613,131.241428
4,2021-02-26 00:00:00+00:00,121.260002,29.668229,-40.663541,-13.870003,-2.076043,141.601533,131.5530,121.504468,130.134286


In [6]:
# Label the data based on the direction of the next day’s price movement
train_data['IsUp'] = (train_data['Close'].shift(-1) > train_data['Close']).astype(int)
test_data['IsUp'] = (test_data['Close'].shift(-1) > test_data['Close']).astype(int)

# Remove the last rows since they have no valid future value
train_data = train_data[:-1].reset_index(drop=True)
test_data = test_data[:-1].reset_index(drop=True)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (218, 11)
Test shape: (217, 11)


In [7]:
train_data.head()

Price,Date,Close,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA,IsUp
Ticker,,AAPL,,,,,,,,,
0,2021-02-22 00:00:00+00:00,126.000000,35.842782,-28.314437,-10.759995,-2.001957,143.672556,135.1415,126.610444,133.678571,0
1,2021-02-23 00:00:00+00:00,125.860001,35.632885,-28.734230,-11.050003,-2.012881,142.948565,134.2885,125.628436,133.026428,0
2,2021-02-24 00:00:00+00:00,125.349998,34.832645,-30.334709,-10.659996,-1.958668,141.887017,133.3980,124.908983,132.412857,0
3,2021-02-25 00:00:00+00:00,120.989998,28.864578,-42.270843,-14.400002,-2.107705,141.477387,132.3445,123.211613,131.241428,1
4,2021-02-26 00:00:00+00:00,121.260002,29.668229,-40.663541,-13.870003,-2.076043,141.601533,131.5530,121.504468,130.134286,1


In [8]:
# Get rid of the multi-index close price column and create a new one
train_data["Close Price"] = train_data["Close"].values
test_data["Close Price"] = test_data["Close"].values

# Set a new column order
required_columns = ["RSI", "CMO", "MOM", "MACD", "BB_Upper", "BB_Middle", "BB_Lower", "SMA", "Close Price", "IsUp"]
new_train = train_data[required_columns].copy()
new_test = test_data[required_columns].copy()

# Flatten Multi-Index columns
new_train.columns = [' '.join(col).strip() for col in new_train.columns.values]
new_test.columns = [' '.join(col).strip() for col in new_test.columns.values]

# Separate the features and target column
train_features = new_train.drop(columns=["IsUp"])
test_features = new_test.drop(columns=["IsUp"])

# Normalize the feature columns
scaler = StandardScaler()
scaler.fit(train_features)
train_features_normalized = scaler.transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Re-attach the 'Target' column to the normalized data
new_train_normalized = pd.DataFrame(train_features_normalized, columns=train_features.columns)
new_train_normalized['IsUp'] = new_train['IsUp'].values

new_test_normalized = pd.DataFrame(test_features_normalized, columns=test_features.columns)
new_test_normalized['IsUp'] = new_test['IsUp'].values

print(f"New train shape: {new_train_normalized.shape}")
print(f"New test shape: {new_test_normalized.shape}")

New train shape: (218, 10)
New test shape: (217, 10)


In [9]:
new_train_normalized.head()

,RSI,CMO,MOM,MACD,BB_Upper,BB_Middle,BB_Lower,SMA,Close Price,IsUp
0,-1.703154,-1.703154,-1.982887,-2.694250,-0.252228,-0.390311,-0.527923,-0.514585,-1.053491,0
1,-1.721267,-1.721267,-2.028279,-2.708584,-0.303521,-0.457198,-0.608991,-0.562697,-1.062740,0
2,-1.790323,-1.790323,-1.967236,-2.637446,-0.378728,-0.527026,-0.668384,-0.607964,-1.096432,0
3,-2.305335,-2.305335,-2.552620,-2.833014,-0.407750,-0.609636,-0.808508,-0.694387,-1.384466,1
4,-2.235984,-2.235984,-2.469665,-2.791466,-0.398954,-0.671701,-0.949439,-0.776067,-1.366629,1


In [10]:
# Create 10x10 Images
# Combine the 10 days of closing prices and technical indicators into a 10x10 matrix
def create_images(data, window_size=10):

    images, labels = list(), list()

    for i in range(len(data) - window_size):
        img = data.iloc[i:(i+window_size)].values
        label = data.iloc[(i+window_size), -1]
        
        if img.shape == (window_size, window_size):
            images.append(img)
            labels.append(label)
        else:
            print(f"Expected a (10, 10) image, but current image shape is: {img.shape}")
    
    return np.array(images), np.array(labels)

# Use the normalized data
train_images, train_labels = create_images(new_train_normalized)
test_images, test_labels = create_images(new_test_normalized)

print(f"Train images: {train_images.shape} & Train labels: {train_labels.shape}")
print(f"Test images: {test_images.shape} & Test labels: {test_labels.shape}")

Train images: (208, 10, 10) & Train labels: (208,)
Test images: (207, 10, 10) & Test labels: (207,)


In [11]:
# Define a CNN model
cnn_model = Sequential([
    # First convolutional layer
    Conv2D(64, (2, 2), activation='relu', input_shape=(10, 10, 1)),
    MaxPooling2D((2, 2)),
    Flatten(), # Flatten the output for the dense layer
    Dense(32, activation='relu'), # Fully connected layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(1, activation='sigmoid') # Output layer (binary classification)
])

# Compile the model with an initial learning rate
cnn_model.compile(
    optimizer=Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=['accuracy'],
)
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 64)          0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                        

In [12]:
# Reshape the images to match the required input shape (10, 10, 1) for the CNN model
final_train_images = train_images.reshape(-1, 10, 10, 1)
final_test_images = test_images.reshape(-1, 10, 10, 1)

# Train the CNN model
history = cnn_model.fit(
    final_train_images, 
    train_labels, 
    epochs=30, 
    batch_size=32, 
    validation_data=(final_test_images, test_labels), 
    verbose=1,
)

Epoch 1/30
7/7 [==============================] - 2s 87ms/step - loss: 0.6975 - accuracy: 0.5048 - val_loss: 0.6937 - val_accuracy: 0.4928
Epoch 2/30
7/7 [==============================] - 0s 30ms/step - loss: 0.6917 - accuracy: 0.5000 - val_loss: 0.6952 - val_accuracy: 0.4879
Epoch 3/30
7/7 [==============================] - 0s 29ms/step - loss: 0.6914 - accuracy: 0.5048 - val_loss: 0.6969 - val_accuracy: 0.4686
Epoch 4/30
7/7 [==============================] - 0s 29ms/step - loss: 0.6906 - accuracy: 0.5481 - val_loss: 0.6961 - val_accuracy: 0.4783
Epoch 5/30
7/7 [==============================] - 0s 60ms/step - loss: 0.6863 - accuracy: 0.5529 - val_loss: 0.6975 - val_accuracy: 0.4734
Epoch 6/30
7/7 [==============================] - 0s 28ms/step - loss: 0.6900 - accuracy: 0.5385 - val_loss: 0.6965 - val_accuracy: 0.4879
Epoch 7/30
7/7 [==============================] - 0s 31ms/step - loss: 0.6848 - accuracy: 0.5865 - val_loss: 0.6968 - val_accuracy: 0.4879
Epoch 8/30
7/7 [===========

In [13]:
loss, accuracy = cnn_model.evaluate(final_test_images, test_labels, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

7/7 [==============================] - 0s 5ms/step - loss: 0.7043 - accuracy: 0.4686
Test Accuracy: 46.86%


**Discussion:** The test accuracy of 46.86% is relatively low, even worse than random guessing, which would yield around 50% accuracy in a binary classification problem. Several factors could explain this performance:
- The technical indicators used may not be sufficient or directly relevant for predicting the target, which is whether the Bitcoin price will go up or down.
- With only 208 training samples, the dataset may be too small to capture meaningful patterns between features and the target variable.
- The basic CNN architecture without hyperparameter tuning may not provide enough capacity or optimization to effectively learn complex relationships in the data.

# END